In [1]:
import codecs
import json
import joblib
import re
from functools import reduce
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import codecs
import json
import joblib
import re
from functools import reduce
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import trange, tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
import pandas as pd
from sklearn.metrics import accuracy_score
import torch.nn as nn
import gc

In [2]:
from tqdm.notebook import trange

In [3]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [4]:
import codecs
import json
def read_jsonl(path):
    with codecs.open(path, encoding='utf-8-sig') as reader:
            lines = reader.read().split("\n")
            lines = list(map(json.loads, filter(None, lines)))
    return lines

In [5]:
def standardize(txt):
    return txt.replace('\n', '').replace('@highlight', '')

In [6]:
def find_closest_string(string, arr, r=0):
    maxx = 0
    most_simillar = None
    for w in arr:
        if w == string:
            return string
        ratio = fuzz.ratio(string, w)
        if ratio > maxx:
            most_simillar = w
            maxx = ratio
    return most_simillar

In [33]:
def to_mr_format(item, translate=True):
    entities = list(set([item["passage"]["text"][e["start"]:e["end"]] for e in item["passage"]["entities"]]))
    en_entities = [answers_en[e] if translate else e for e in entities]
    for e in en_entities:
        translated_text = texts_en[standardize(item["passage"]["text"])] if translate else item["passage"]["text"]
        translated_text = "".join([f"Sent {i + 1}: " + s + "." for i, s in enumerate(split_into_sentences(translated_text))])
        tanslated_query = questions_en[standardize(item["qas"][0]["query"])] if translate else questions_en[item["qas"][0]["query"]]
        final_text = "multirc question: " + tanslated_query + " answer: " + e + " paragraph: " + translated_text
        #final_text = "record query: " + (tanslated_query + ". entities: " if tanslated_query[-1] != "." else tanslated_query + " entities: ") + ", ".join(en_entities) + " passage: " + (translated_text + "." if translated_text[-1] != "." else translated_text)
        return final_text

def to_mr_format_all(item, test=False):
    entities = list(set([item["passage"]["text"][e["start"]:e["end"]] for e in item["passage"]["entities"]]))
    correct_answers = [i["text"] for i in item["qas"][0]["answers"]] if not test else []
    labels = []
    en_entities = []
    for extracted in entities:
        if not test:
            labels.append( int(extracted in correct_answers) )
        else:
            labels.append(None)
        en_entities.append( answers_en_all[ extracted ] )

    final_texts = []
    for e in en_entities:
        translated_text = texts_en_all[ standardize(item["passage"]["text"]) ] 
        translated_text = "".join([f"Sent {i + 1}: " + s + "." for i, s in enumerate(split_into_sentences(translated_text))])
        tanslated_query = questions_en_all[ standardize(item["qas"][0]["query"]) ] 
        final_text = "multirc question: " + tanslated_query + " answer: " + e + " paragraph: " + translated_text
        #final_text = "record query: " + (tanslated_query + ". entities: " if tanslated_query[-1] != "." else tanslated_query + " entities: ") + ", ".join(en_entities) + " passage: " + (translated_text + "." if translated_text[-1] != "." else translated_text)
        final_texts.append(final_text)

    return final_texts, labels

def priotarization(item):
    entities = list(set([item["passage"]["text"][e["start"]:e["end"]] for e in item["passage"]["entities"]]))
    correct_answers = [i["text"] for i in item["qas"][0]["answers"]]
    labels = []
    en_entities = []
    for extracted in entities:
        labels.append( int(extracted in correct_answers) )
        en_entities.append( answers_en_all[ extracted ] )

    translated_text = texts_en[standardize(item["passage"]["text"])] if translate else item["passage"]["text"]
    tanslated_query = questions_en[standardize(item["qas"][0]["query"])] if translate else item["passage"]["text"]
    final_text = "record query: " + (tanslated_query + ". entities: " if tanslated_query[-1] != "." else tanslated_query + " entities: ") + "{}" + " passage: " + (translated_text + "." if translated_text[-1] != "." else translated_text)

    return final_text, en_entities

def split_X_y(data):
    X = []
    y = []
    for d in data:
      X.extend(d[0])
      y.extend(d[1])
    return X, y

def fillthegaps():
    global texts_en_all
    texts_en_all['Обвинения в причастности к парижским терактам предъявлены в Бельгии уже восьми задержанным. Один из организаторов взрывов был перед терактами в Будапеште. В Бельгии задержаны еще два подозреваемых в связи с терактами в Париже, один из них гражданин Франции, второй - подданный Бельгии. Их обвиняют в участии в действиях, связанных с терроризмом, сообщили в четверг, 3 декабря, в генпрокуратуре в Брюсселе. Они останутся под арестом в течение месяца. Олланд постарался убедить мировых лидеров в необходимости более интенсивной борьбы с ИГ.В Германии за сутки выявлено более 100 новых заражений коронавирусомТысячи демонстрантов в Гамбурге выступили за прием беженцевКомментарий: Россия накануне эпидемии - виноватые назначены заранее'] = "Already eight detainees have been charged with involvement in the Paris terrorist attacks in Belgium. One of the organizers of the explosions was before the attacks in Budapest. In Belgium, two more suspects were detained in connection with the terrorist attacks in Paris, one of them is a citizen of France, the other is a citizen of Belgium. They are accused of participation in acts related to terrorism, reported on Thursday, December 3, at the Prosecutor General's Office in Brussels. They will remain under arrest for a month. Hollande tried to convince world leaders of the need for a more intense fight against IS. More than 100 new coronavirus infections were detected in Germany per day Thousands of demonstrators in Hamburg spoke in favor of accepting refugees."
    texts_en_all['В Саитаме (Япония) пожилую женщину госпитализировали с подозрением на заражение вирусом Эбола. Об этом сообщает РИА Новости со ссылкой на агентство Киодо. Уточняется, что ранее 70-летняя японка посещала Демократическую Республику Конго. У нее была зафиксирована температура более 38 градусов. Пенсионерку доставили в одну из больниц Токио для выявления в ее крови вируса Эбола. В кризисном штабе при канцелярии премьер-министра Японии Синдзо Абэ уже была создана группа по сбору и анализу информации. Смертельный вирус достиг международного масштаба Смертельный вирус проник в еще одну страну Африки «Русал» и Минздрав завершили вакцинацию против Эболы в Гвинее'] = "In Saitama, Japan, an elderly woman was hospitalized with suspected Ebola infection. This was reported by RIA Novosti with reference to the Kyodo agency. It is specified that the 70-year-old Japanese woman previously visited the Democratic Republic of the Congo. She had a temperature of more than 38 degrees. The pensioner was taken to a hospital in Tokyo to be diagnosed with the Ebola virus in her blood. A group for collecting and analyzing information has already been set up in the crisis headquarters under the office of Japanese Prime Minister Shinzo Abe. The deadly virus has reached an international scale The deadly virus has penetrated another African country Rusal and the Ministry of Health completed vaccination against Ebola in Guinea"
    texts_en_all['Физики из Университета Глазго обнаружили парадокс теории Эйнштейна, который может нарушать основополагающий принцип равноправия инерциальных систем отсчета (ИСО) и тем самым ставит под угрозу физическую картину мира. Объяснение противоречия было предложено в Journal of Modern Optics. Кратко об исследовании рассказывает издание Gizmodo. Согласно принципу относительности для разных ИСО, представленных покоящимися и двигающимися телами, законы физики проявляются одинаково. Однако ученые выяснили, что этот принцип может не выполняться для движущихся возбужденных атомов. Парадокс заключается в том, что атомы могут спонтанно испускать фотоны, но при этом на частицы света распространяется эффект Доплера. Представлено новое подтверждение общей теории относительности Существование машины времени доказали математически Физики запутали рекордное количество атомов'] = "Physicists from the University of Glasgow discovered the paradox of Einstein's theory, which can violate the fundamental principle of equality of inertial reference systems (IRF) and thereby endanger the physical picture of the world. An explanation for the controversy was suggested in the Journal of Modern Optics. The research is briefly described by the publication Gizmodo. According to the principle of relativity for different IFRs, represented by bodies at rest and moving bodies, the laws of physics manifest themselves in the same way. However, scientists have found that this principle may not hold for moving excited atoms. The paradox is that atoms can spontaneously emit photons, but the Doppler effect applies to light particles. New confirmation of general relativity presented The existence of a time machine has been proven mathematically Physicists have entangled a record number of atoms"
    texts_en_all['Бывший президент Франции Николя Саркози подверг критике подход нынешнего главы государства Франсуа Олланда к отношениям с Москвой. Его слова приводит ТАСС. «Я сожалею о политике, которая проводится ныне в отношении России», — заявил он журналистам, комментируя известие об отмене визита в Париж президента Владимира Путина. Саркози, намеревающийся баллотироваться в 2017 году на пост президента, предостерег от «вступления в новую холодную войну». По его словам, «долг и обязанность Франции и Европы — вести диалог с Россией». «Как можно урегулировать ситуацию, если общаться друг с другом лишь посредством пресс-релизов?» — сказал Саркози.Олланд отказался считать отмену визита Путина помехой для дискуссии по СирииВо Франции сообщили об отмене визита Путина в ПарижПесков объяснил отмену визита Путина в Париж'] = """Former French President Nicolas Sarkozy criticized the current head of state François Hollande's approach to relations with Moscow. His words are quoted by TASS. “I regret the current policy towards Russia,” he told reporters, commenting on the news of the cancellation of President Vladimir Putin’s visit to Paris. Sarkozy, intending to run for president in 2017, has warned against "entering a new cold war." According to him, "the duty and obligation of France and Europe is to conduct a dialogue with Russia." "How can you resolve the situation if you communicate with each other only through press releases?" - Sarkozy said. Hollande refused to consider the cancellation of Putin's visit as a hindrance to the discussion on Syria France announced the cancellation of Putin's visit to Paris Sands explained the cancellation of Putin's visit to Paris"""
    texts_en_all['Президент России Владимир Путин заявил, что выступает за бессрочный закон об особом статусе Донбасса. Об этом он заявил на пресс-конференции после переговоров с канцлером Германии Ангелой Меркель в Кремле, передает «Интерфакс». Глава государства призвал руководство Украины начать реализацию минских соглашений, несмотря на внутриполитические проблемы. «Мы понимаем все сложности внутриполитического процесса на Украине, но, если все заинтересованные стороны хотят добиться окончательного урегулирования, нужно идти по реализации минских соглашений», — сказал Путин. Он также отметил, что на переговорах с канцлером Германии «подробно обсуждалось» урегулирование внутриукраинского кризиса. Ранее министр иностранных дел Украины Вадим Пристайко заявил, что принятие «формулы Штайнмайера» не означает проведения выборов в Донбассе любой ценой.Украина отказалась проводить выборы в Донбассе любой ценойОписаны пять сценариев развития ситуации в ДонбассеНа Украине раскрыли план по введению «формулы Штайнмайера» в законодательство'] = """Russian President Vladimir Putin said he was in favor of an unlimited law on the special status of Donbass. He stated this at a press conference after talks with German Chancellor Angela Merkel in the Kremlin, Interfax reports. The head of state called on the Ukrainian leadership to start implementing the Minsk agreements, despite the internal political problems. “We understand all the complexities of the internal political process in Ukraine, but if all interested parties want to achieve a final settlement, we need to follow the implementation of the Minsk agreements,” Putin said. He also noted that during the talks with the German Chancellor, the settlement of the internal Ukrainian crisis was "discussed in detail". Earlier, the Minister of Foreign Affairs of Ukraine Vadim Prystaiko said that the adoption of the "Steinmeier formula" does not mean holding elections in Donbass at any cost."""





In [8]:
def fix_placeholder(string):
  return string.replace('@Placeholder', '@placeholder').replace('@ placeholder', '@placeholder').replace('@ Placeholder', '@placeholder')

In [9]:
!pip install fuzzywuzzy

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


In [10]:
import json
from fuzzywuzzy import fuzz

/home/user/conda/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
!pip install transformers

  Using cached transformers-4.3.3-py3-none-any.whl (1.9 MB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached tokenizers-0.10.1-cp37-cp37m-manylinux2010_x86_64.whl (3.2 MB)
Processing /home/jovyan/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0/sacremoses-0.0.43-py3-none-any.whl


In [13]:
answers_en = json.loads(open("answers_all.json").read())
questions_en = json.loads(open("questions_test.json").read())
texts_en = json.loads(open("texts_test.json").read())

In [12]:
texts_en_all = json.loads(open("texts_validation.json").read())
answers_en_all = json.loads(open("answers_all.json").read())
questions_en_all = json.loads(open("questions_validation.json").read())

In [14]:
train = read_jsonl("rucos_train.jsonl")
val = read_jsonl("rucos_val.jsonl")
test = read_jsonl("rucos_test.jsonl")

In [15]:
from sklearn.metrics import f1_score

In [17]:
fillthegaps()

In [18]:
multirc_val = list(map(to_mr_format_all, val))

In [30]:
texts_en_all = json.loads(open("texts_test.json").read())
questions_en_all = json.loads(open("questions_test.json").read())

In [34]:
multirc_test = list(map((lambda x: to_mr_format_all(x, test=True)), test))

In [36]:
X_test, y_test = split_X_y(multirc_test)

In [22]:
X_val = list(map(fix_placeholder, X_val))

In [39]:
X_test = list(map(fix_placeholder, X_test))

In [40]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
model = T5ForConditionalGeneration.from_pretrained('t5-3b')
tokenizer = T5TokenizerFast.from_pretrained('t5-3b')

In [41]:
torch.cuda.empty_cache()
gc.collect()

1059

In [42]:
model.parallelize()

In [34]:
val_tokens = tokenizer(list(X_val), padding=True, truncation=True, return_tensors="pt").input_ids

In [ ]:
val_preds = []
batch_size = 8
for i in trange(0,len(val_tokens),batch_size):
    outputs = model.generate(torch.tensor(val_tokens[i:i+batch_size]).to("cuda"))
    preds = 1 - outputs.cpu().numpy()[:,-1]
    val_preds.extend(list(preds))
    if i % 64 == 0:
        torch.cuda.empty_cache()
        gc.collect()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [43]:
test_tokens = tokenizer(list(X_test), padding=True, truncation=True, return_tensors="pt").input_ids

In [ ]:
test_preds = []
batch_size = 64
for i in range(0,len(test_tokens),batch_size):
    outputs = model.generate(torch.tensor(test_tokens[i:i+batch_size]).to("cuda"))
    preds = 1 - outputs.cpu().numpy()[:,-1]
    test_preds.extend(list(preds))
    if i % 64 == 0:
        torch.cuda.empty_cache()
        gc.collect()
        print(i)

In [46]:
import numpy as np

In [47]:
np.save("multircformat_test_first8832.npy", np.array(test_preds))

In [91]:
np.save("multirc_record_base.npy", np.array(val_preds))

In [87]:
ids = [i for i in range(len(y_val)) if y_val[i]]

In [ ]:
mp.array(y_val)[ids]

In [ ]:
f1_score(np.array(val_preds)[ids], np.array(y_val)[ids])